In [84]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math

data_size = 100000
train_portion = 0.9999
train_size = (int)(data_size * train_portion)
X = (np.random.ranf((data_size, 2)) * 2 - 1) / math.pi
y = np.zeros((data_size, 1))
for i in xrange(data_size):
    # linear function
    y[i] = 2 * X[i, 0] - 1 * X[i, 1]
    # nonlinear function
#     y[i] = 0.5 * math.pi * (X[i, 0] ** 2) * math.sin(2 * math.pi * X[i, 1])   

X_train = X[: train_size]
y_train = y[: train_size]
X_test = X[train_size:]
y_test = y[train_size:]

batch_size = 10000
input_size = 2
hidden_size = 15
output_size = 1

graph = tf.Graph()

with graph.as_default():
    # prepare the data
    std = 1
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, input_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, output_size))   
    tf_test_dataset = tf.constant(X_test, dtype=tf.float32)
    
    layer1_weights = std * tf.Variable(tf.truncated_normal(
        [input_size, hidden_size], stddev=1))
    layer1_biases = tf.Variable(tf.zeros([hidden_size]))
    
    layer2_weights = std * tf.Variable(tf.truncated_normal(
        [hidden_size, output_size], stddev=1))
    layer2_biases = tf.Variable(tf.zeros([output_size]))
    
    # define the model
    def model(data):
        hidden = tf.sigmoid(tf.matmul(data, layer1_weights) + layer1_biases)
        
        result = tf.matmul(hidden, layer2_weights) + layer2_biases
        
        return result
    
    # define the loss
    logits = model(tf_train_dataset)
    loss = tf.reduce_sum(tf.squared_difference(logits, tf_train_labels)) / train_size
    # learning rate decay
    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = 1e-2
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           100000, 0.96, staircase=True)
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    train_prediction = logits
    test_prediction = model(tf_test_dataset)

In [11]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [12]:
show_graph(graph)

In [85]:
num_steps = 2000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (y_train.shape[0] - batch_size)
        batch_data = X_train[offset:(offset + batch_size), :]
        batch_labels = y_train[offset:(offset + batch_size), ]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
    print test_prediction.eval(session=session)
    
    print y_test

Initialized
Minibatch loss at step 0: 1.041763
Minibatch loss at step 50: 0.323288
Minibatch loss at step 100: 0.103439
Minibatch loss at step 150: 0.034474
Minibatch loss at step 200: 0.012243
Minibatch loss at step 250: 0.005358
Minibatch loss at step 300: 0.003041
Minibatch loss at step 350: 0.002345
Minibatch loss at step 400: 0.002098
Minibatch loss at step 450: 0.002000
Minibatch loss at step 500: 0.001955
Minibatch loss at step 550: 0.001924
Minibatch loss at step 600: 0.001850
Minibatch loss at step 650: 0.001908
Minibatch loss at step 700: 0.001829
Minibatch loss at step 750: 0.001788
Minibatch loss at step 800: 0.001819
Minibatch loss at step 850: 0.001795
Minibatch loss at step 900: 0.001780
Minibatch loss at step 950: 0.001758
Minibatch loss at step 1000: 0.001725
Minibatch loss at step 1050: 0.001661
Minibatch loss at step 1100: 0.001711
Minibatch loss at step 1150: 0.001647
Minibatch loss at step 1200: 0.001609
Minibatch loss at step 1250: 0.001641
Minibatch loss at step 